In [1]:
!pip install googletrans==4.0.0-rc1
# !pip install adapter-transformers
!pip install deep-translator
# !pip uninstall transformers -y
# !pip install transformers
!pip install adapters
!pip install transformers sentencepiece


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=2626de49c0be87645cbf6b9d2398b86831b62c77debed1023c288a43921a510b
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfra

In [2]:
import pandas as pd
import re
from sklearn.preprocessing import MultiLabelBinarizer
import torch
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    Trainer
)

from adapters import (
    AutoAdapterModel,
    AdapterTrainer,
    AdapterArguments
)
from googletrans import Translator
import pandas as pd
import time

Afrikaans pre-processing


In [3]:
#### AFRIKAANS ####
# Login using e.g. `huggingface-cli login` to access this dataset
# Split the dataset
splits = {'train': 'afr/train-00000-of-00001.parquet', 'dev': 'afr/dev-00000-of-00001.parquet', 'test': 'afr/test-00000-of-00001.parquet'}

# Training df
df_train_afr = pd.read_parquet("hf://datasets/brighter-dataset/BRIGHTER-emotion-categories/" + splits["train"])
df_train_afr.head()

# Dev df
df_dev_afr = pd.read_parquet("hf://datasets/brighter-dataset/BRIGHTER-emotion-categories/" + splits["dev"])
df_dev_afr.head()

# Testing df
df_test_afr = pd.read_parquet("hf://datasets/brighter-dataset/BRIGHTER-emotion-categories/" + splits["test"])
df_test_afr.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,id,text,anger,disgust,fear,joy,sadness,surprise,emotions
0,afr_test_track_a_00001,as regering is ons daartoe verbind om ons deel...,0,0,0,1,0,NaN,[joy]
1,afr_test_track_a_00002,op die oomblik is die kwessie van voedselsekur...,0,0,1,0,1,NaN,"[fear, sadness]"
2,afr_test_track_a_00003,ek hoor dikwels mense sê hulle is gereed om be...,0,0,0,0,0,NaN,[]
3,afr_test_track_a_00004,hiervan bly kindermishandeling waarskynlik een...,1,1,0,0,1,NaN,"[anger, disgust, sadness]"
4,afr_test_track_a_00005,so gaan ons ernstig kyk na die kwaliteit van o...,0,0,0,0,0,NaN,[]


In [4]:
#### AFRIKAANS ####
# Structure of the dataset
print(df_train_afr.columns)
print(df_train_afr.iloc[0])

Index(['id', 'text', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise',
       'emotions'],
      dtype='object')
id                                    afr_train_track_a_00001
text        die grondeienaars het die departement genader ...
anger                                                       0
disgust                                                     0
fear                                                        0
joy                                                         0
sadness                                                     0
surprise                                                  NaN
emotions                                                   []
Name: 0, dtype: object


In [5]:
#### AFRIKAANS ####
# Check for empty rows
empty_text_rows = df_train_afr['text'].str.strip().eq('')
print(f"Empty text rows: {empty_text_rows.sum()}")

# Check for NaN
print(f"NaN: {df_train_afr.isnull().sum()}")

# Fill 'surprise' column with '0'
df_train_afr['surprise'] = df_train_afr['surprise'].fillna(0)
df_dev_afr['surprise'] = df_dev_afr['surprise'].fillna(0)
df_test_afr['surprise'] = df_test_afr['surprise'].fillna(0)

Empty text rows: 0
NaN: id             0
text           0
anger          0
disgust        0
fear           0
joy            0
sadness        0
surprise    1222
emotions       0
dtype: int64


In [6]:
#### AFRIKAANS ####
# Remove duplicates based on 'text' column
print("Duplicate texts in train:", df_train_afr['text'].duplicated().sum())
print("Duplicate texts in dev:", df_dev_afr['text'].duplicated().sum())
print("Duplicate texts in test:", df_test_afr['text'].duplicated().sum())

# Keep only first occurence of duplicate
df_train_afr = df_train_afr.drop_duplicates(subset='text')
df_dev_afr = df_dev_afr.drop_duplicates(subset='text')
df_test_afr = df_test_afr.drop_duplicates(subset='text')

Duplicate texts in train: 0
Duplicate texts in dev: 98
Duplicate texts in test: 1065


In [7]:
#### AFRIKAANS ####
# Remove punctuation and spaces
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_train_afr['text'] = df_train_afr['text'].astype(str).apply(clean_text)
df_dev_afr['text'] = df_dev_afr['text'].astype(str).apply(clean_text)
df_test_afr['text'] = df_test_afr['text'].astype(str).apply(clean_text)

/tmp/ipython-input-7-2030011785.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dev_afr['text'] = df_dev_afr['text'].astype(str).apply(clean_text)


Swahili pre-processing


In [8]:
#### SWAHILI ####
# Split the data
# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'train': 'swa/train-00000-of-00001.parquet', 'dev': 'swa/dev-00000-of-00001.parquet', 'test': 'swa/test-00000-of-00001.parquet'}

# Training df
df_train_swa = pd.read_parquet("hf://datasets/brighter-dataset/BRIGHTER-emotion-categories/" + splits["train"])

# Dev df
df_dev_swa = pd.read_parquet("hf://datasets/brighter-dataset/BRIGHTER-emotion-categories/" + splits["dev"])

# Testing df
df_test_swa = pd.read_parquet("hf://datasets/brighter-dataset/BRIGHTER-emotion-categories/" + splits["test"])


In [9]:
#### SWAHILI ####
# Structure of the dataset
print(df_train_swa.columns)
print(df_train_swa.iloc[0])

Index(['id', 'text', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise',
       'emotions'],
      dtype='object')
id                               swa_train_track_a_00001
text        hii game ni goals zimetukataa tu kumamaye 😭💔
anger                                                  1
disgust                                                0
fear                                                   0
joy                                                    0
sadness                                                0
surprise                                               0
emotions                                         [anger]
Name: 0, dtype: object


In [10]:
#### SWAHILI ####
# Check for empty rows
empty_text_rows = df_train_swa['text'].str.strip().eq('')
print(f"Empty text rows: {empty_text_rows.sum()}")

# Check for NaN
print(f"NaN: {df_train_swa.isnull().sum()}")

Empty text rows: 0
NaN: id          0
text        0
anger       0
disgust     0
fear        0
joy         0
sadness     0
surprise    0
emotions    0
dtype: int64


In [11]:
#### SWAHILI ####
# Check for duplicate texts
print("Duplicate texts in Swahili train:", df_train_swa['text'].duplicated().sum())
print("Duplicate texts in Swahili dev:", df_dev_swa['text'].duplicated().sum())
print("Duplicate texts in Swahili test:", df_test_swa['text'].duplicated().sum())

# Keep first occurence of duplicate
df_train_swa = df_train_swa.drop_duplicates(subset='text')
df_dev_swa = df_dev_swa.drop_duplicates(subset='text')
df_test_swa = df_test_swa.drop_duplicates(subset='text')

Duplicate texts in Swahili train: 27
Duplicate texts in Swahili dev: 551
Duplicate texts in Swahili test: 1664


In [12]:
#### SWAHILI ####
# Remove punctuation and spaces
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_train_swa['text'] = df_train_swa['text'].astype(str).apply(clean_text)
df_dev_swa['text'] = df_dev_swa['text'].astype(str).apply(clean_text)
df_test_swa['text'] = df_test_swa['text'].astype(str).apply(clean_text)

/tmp/ipython-input-12-2316263755.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_swa['text'] = df_train_swa['text'].astype(str).apply(clean_text)
/tmp/ipython-input-12-2316263755.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dev_swa['text'] = df_dev_swa['text'].astype(str).apply(clean_text)


English pre-processing


In [13]:
#### ENGLISH ####
# Login using e.g. `huggingface-cli login` to access this dataset
# Split the dataset

splits = {
    'train': 'eng/train-00000-of-00001.parquet',
    'dev': 'eng/dev-00000-of-00001.parquet',
    'test': 'eng/test-00000-of-00001.parquet'
}

# Training df
df_train_eng = pd.read_parquet("hf://datasets/brighter-dataset/BRIGHTER-emotion-categories/" + splits["train"])

# Dev df
df_dev_eng = pd.read_parquet("hf://datasets/brighter-dataset/BRIGHTER-emotion-categories/" + splits["dev"])

#Testing df
df_test_eng = pd.read_parquet("hf://datasets/brighter-dataset/BRIGHTER-emotion-categories/" + splits["test"])


In [14]:
#### ENGLISH ####
empty_text_rows = df_train_eng['text'].str.strip().eq('')
print(f"Empty text rows: {empty_text_rows.sum()}")

# Check for NaN
print(f"NaN: {df_train_eng.isnull().sum()}")

# Fill 'disgust' column with '0'
df_train_eng['disgust'] = df_train_eng['disgust'].fillna(0)
df_dev_eng['disgust'] = df_dev_eng['disgust'].fillna(0)
df_test_eng['disgust'] = df_test_eng['disgust'].fillna(0)

Empty text rows: 0
NaN: id             0
text           0
anger          0
disgust     2768
fear           0
joy            0
sadness        0
surprise       0
emotions       0
dtype: int64


In [15]:
#### ENGLISH ####
# Check for duplicate texts
print("Duplicate texts in English train:", df_train_eng['text'].duplicated().sum())
print("Duplicate texts in English dev:", df_dev_eng['text'].duplicated().sum())
print("Duplicate texts in English test:", df_test_eng['text'].duplicated().sum())

# Keep first occurence of duplicate
df_train_eng = df_train_eng.drop_duplicates(subset='text')
df_dev_eng = df_dev_eng.drop_duplicates(subset='text')
df_test_eng = df_test_eng.drop_duplicates(subset='text')

Duplicate texts in English train: 4
Duplicate texts in English dev: 116
Duplicate texts in English test: 2772


In [16]:
#### ENGLISH ####
# Remove punctuation and spaces
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_train_eng['text'] = df_train_eng['text'].astype(str).apply(clean_text)
df_dev_eng['text'] = df_dev_eng['text'].astype(str).apply(clean_text)
df_test_eng['text'] = df_test_eng['text'].astype(str).apply(clean_text)

Tokenization

In [17]:
# Tokenization
def tokenize_texts(texts, tokenizer, max_length=128):
    return tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# Afrikaan
train_encodings_afr = tokenize_texts(df_train_afr['text'], tokenizer)
dev_encodings_afr = tokenize_texts(df_dev_afr['text'], tokenizer)
test_encodings_afr = tokenize_texts(df_test_afr['text'], tokenizer)

# Swahili
train_encodings_swa = tokenize_texts(df_train_swa['text'], tokenizer)
dev_encodings_swa = tokenize_texts(df_dev_swa['text'], tokenizer)
test_encodings_swa = tokenize_texts(df_test_swa['text'], tokenizer)

# English
train_encodings_eng = tokenize_texts(df_train_eng['text'], tokenizer)
dev_encodings_eng = tokenize_texts(df_dev_eng['text'], tokenizer)
test_encodings_eng = tokenize_texts(df_test_eng['text'], tokenizer)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [18]:
# Prepare labels
emotion_cols = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Afrikaans
y_train = df_train_afr[emotion_cols].values
y_dev = df_dev_afr[emotion_cols].values
y_test = df_test_afr[emotion_cols].values

# Swahili
y_train_swa = df_train_swa[emotion_cols].values
y_dev_swa = df_dev_swa[emotion_cols].values
y_test_swa = df_test_swa[emotion_cols].values

# English
y_train_eng = df_train_eng[emotion_cols].values
y_dev_eng = df_dev_eng[emotion_cols].values
y_test_eng = df_test_eng[emotion_cols].values



In [19]:
class BRIGHTEREmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.float32)  # float32 for BCEWithLogitsLoss

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)


In [20]:
# Create pytorch datasets

# Afrikaans
train_dataset_afr = BRIGHTEREmotionDataset(train_encodings_afr, y_train)
dev_dataset_afr = BRIGHTEREmotionDataset(dev_encodings_afr, y_dev)
test_dataset_afr = BRIGHTEREmotionDataset(test_encodings_afr, y_test)

# Swahili
train_dataset_swa = BRIGHTEREmotionDataset(train_encodings_swa, y_train_swa)
dev_dataset_swa = BRIGHTEREmotionDataset(dev_encodings_swa, y_dev_swa)
test_dataset_swa = BRIGHTEREmotionDataset(test_encodings_swa, y_test_swa)

# English
train_dataset_eng = BRIGHTEREmotionDataset(train_encodings_eng, y_train_eng)
dev_dataset_eng = BRIGHTEREmotionDataset(dev_encodings_eng, y_dev_eng)
test_dataset_eng = BRIGHTEREmotionDataset(test_encodings_eng, y_test_eng)


In [21]:
translator = Translator()

# English to Swahili
result = translator.translate("I am happy", src='en', dest='sw')
print(result.text)

# Swahili to English
result = translator.translate("Ninafuraha", src='sw', dest='en')
print(result.text)


Nina furaha
I'm happy


In [22]:
translator = Translator()

def back_translate_text(text, src='en', mid='sw', delay=1.0):
    try:
        translated = translator.translate(text, src=src, dest=mid).text
        back_translated = translator.translate(translated, src=mid, dest=src).text
        time.sleep(delay)
        return back_translated
    except Exception as e:
        print(f"Translation failed for: {text} — {e}")
        return text


In [23]:
# Back-translate first N examples for speed
N = 100
df_bt = df_train_eng.iloc[:N].copy()
df_bt["text"] = df_bt["text"].apply(back_translate_text)

# Combine with original training set
df_augmented = pd.concat([df_train_eng, df_bt])
df_augmented = df_augmented.drop_duplicates(subset="text").reset_index(drop=True)


In [24]:
import torch
from torch.utils.data import Dataset

class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).float()
        return item

    def __len__(self):
        return len(self.labels)

train_encodings_eng_aug = tokenize_texts(df_augmented['text'], tokenizer)
train_labels_eng_aug = df_augmented[['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']].values

train_dataset_eng_aug = EmotionDataset(train_encodings_eng_aug, train_labels_eng_aug)

# Need to also wrap dev dataset in the same class
dev_dataset_eng = EmotionDataset(dev_encodings_eng, y_dev_eng)

In [25]:
###
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import torch

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=6,  # anger, disgust, fear, joy, sadness, surprise
    problem_type="multi_label_classification"
)

training_args = TrainingArguments(
    output_dir="./results_crosslingual",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from torch.utils.data import Subset

def compute_metrics(pred):
    logits, labels = pred
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    preds = (probs >= 0.5).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='macro', zero_division=0
    )
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# subset_indices = list(range(16))  # first 16 samples

# small_train_dataset = Subset(train_dataset_eng_aug, subset_indices)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_eng_aug,
    # train_dataset=small_train_dataset,
    eval_dataset=dev_dataset_eng,
    compute_metrics=compute_metrics
)


trainer.train()

trainer.save_model("mbert_emotion_crosslingual")
tokenizer.save_pretrained("mbert_emotion_crosslingual")

print("Evaluating on Afrikaans...")
afr_metrics = trainer.evaluate(eval_dataset=test_dataset_afr)
print("Afrikaans:", afr_metrics)

print("Evaluating on Swahili...")
swa_metrics = trainer.evaluate(eval_dataset=test_dataset_swa)
print("Swahili:", swa_metrics)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


...

/tmp/ipython-input-24-589065473.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.504000,0.465535,0.146552,0.173533,0.197365,0.204301
2,0.427500,0.399478,0.379310,0.404850,0.485757,0.365455
3,0.365100,0.385015,0.310345,0.376620,0.460339,0.350384
4,0.329700,0.376859,0.379310,0.426902,0.535383,0.399937


/tmp/ipython-input-24-589065473.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipython-input-24-589065473.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipython-input-24-589065473.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Evaluating on Afrikaans...


Afrikaans: {'eval_loss': 0.49808916449546814, 'eval_accuracy': 0.12863849765258217, 'eval_f1': 0.1573475196411647, 'eval_precision': 0.4139646290330828, 'eval_recall': 0.17620455265913093, 'eval_runtime': 6.8946, 'eval_samples_per_second': 154.468, 'eval_steps_per_second': 4.931, 'epoch': 4.0}
Evaluating on Swahili...
Swahili: {'eval_loss': 0.47465604543685913, 'eval_accuracy': 0.12014563106796117, 'eval_f1': 0.10978720278348346, 'eval_precision': 0.13343576370803287, 'eval_recall': 0.21398082755154654, 'eval_runtime': 10.8489, 'eval_samples_per_second': 151.905, 'eval_steps_per_second': 4.793, 'epoch': 4.0}


In [26]:
print("Evaluating on English...")
eng_metrics = trainer.evaluate(eval_dataset=test_dataset_eng)
print("English:", eng_metrics)

Evaluating on English...


English: {'eval_loss': 0.38579240441322327, 'eval_accuracy': 0.3392469225199131, 'eval_f1': 0.42898500498720793, 'eval_precision': 0.5761912496071847, 'eval_recall': 0.39834611197551784, 'eval_runtime': 18.7064, 'eval_samples_per_second': 147.65, 'eval_steps_per_second': 4.651, 'epoch': 4.0}
